In [2]:
model_name = 'sparse_tfidf'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp
import gc

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
df = pd.read_csv(data_dir + 'preprocessed.csv') 
q = pd.read_csv(data_dir + 'sessions.csv')
df = df.merge(q, on='uid', how='left')
del q
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

mat1 = sp.load_npz(data_dir+'dmat1.npz').tolil()
mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()

mat = sp.hstack([mat1,mat2,mat3]).tolil()
del mat1,mat2,mat3

train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]
mat = mat.tocsc()[:, np.where((train_mat.getnnz(axis=0) > 1) & (test_mat.getnnz(axis=0) > 1))[0]].tocsr()

import tqdm
idcs = []
for q in tqdm.tqdm(range(len(df))):
    idcs.append(mat[q].nonzero()[1] + 1)
df['idcs'] = np.array(idcs)
del idcs
gc.collect()

100%|██████████| 609018/609018 [01:26<00:00, 7038.21it/s]


7

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
del mat
gc.collect()

In [5]:
df['idcs'] = df.idcs.apply(lambda x : ' '.join([str(q) for q in x.tolist()]))
mat = TfidfVectorizer(max_features=5000).fit_transform(df.idcs)

In [6]:
X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [7]:
%%time
train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]

CPU times: user 24.5 s, sys: 456 ms, total: 24.9 s
Wall time: 24.4 s


In [10]:
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    'max_depth' : 12,
    #'min_data' : 30,
    'lambda_l2' : 1.5,
    #'min_sum_hessian_in_leaf' : 0.3,
    'lambda_l1' : 12.5,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.7,
    'verbose': 0
}

kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):       
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, train_mat[train_index]
    ]).tocsr()
    print('prepare valid')
    X_va = sp.hstack([
        X_va, train_mat[test_index]
    ]).tocsr()
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = sp.hstack([
        X_te, test_mat
    ]).tocsr()   
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 0
prepare train
prepare valid
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's auc: 0.614857
[100]	valid_0's auc: 0.621345
[150]	valid_0's auc: 0.628203
[200]	valid_0's auc: 0.6361
[250]	valid_0's auc: 0.64317
[300]	valid_0's auc: 0.649981
[350]	valid_0's auc: 0.65662
[400]	valid_0's auc: 0.662531
[450]	valid_0's auc: 0.666509
[500]	valid_0's auc: 0.669892
[550]	valid_0's auc: 0.672308
[600]	valid_0's auc: 0.674164
[650]	valid_0's auc: 0.675304
[700]	valid_0's auc: 0.676457
[750]	valid_0's auc: 0.677398
[800]	valid_0's auc: 0.677962
[850]	valid_0's auc: 0.678636
[900]	valid_0's auc: 0.679115
[950]	valid_0's auc: 0.679698
[1000]	valid_0's auc: 0.679952
[1050]	valid_0's auc: 0.679947
[1100]	valid_0's auc: 0.68012
[1150]	valid_0's auc: 0.680224
[1200]	valid_0's auc: 0.680405
[1250]	valid_0's auc: 0.680337
[1300]	valid_0's auc: 0.680239
[1350]	valid_0's auc: 0.680177
Early stopping, best iteration is:
[1181]	valid_0's auc: 0.680442
0 0.680441969449
prepare

In [12]:
print(scores,np.mean(scores))
roc_auc_score(X.target.values, y_oof.target.values)

[0.6804419694490953, 0.67683422838383911, 0.67673263268769868, 0.68488798864761247, 0.6797403623757039] 0.679727436309


0.6795978050428465

In [ ]:
[0.680441969449, 0.676834228384, 0.676732632688, 0.684887988648]

In [13]:
model_name = 'tfifg_5folds'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

In [14]:
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.051926
1,ac4b8244f3ae82df511b002257473c11,0.034593
2,483d8b91e49522c8a5bbe37f3872c749,0.052588
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.017122
4,fdbfba9842ff0bf86d600eb334c7c42b,0.022017


In [15]:
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)

In [16]:
mat = TfidfVectorizer().fit_transform(df.idcs)
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components = 300, algorithm='arpack')
mat = pca.fit_transform(mat.astype(np.float32))
train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]

In [17]:
np.save(data_dir+'svd_tfidf300.npy',mat)

In [ ]:
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    #'max_depth' : 12,
    #'min_data' : 30,
    'lambda_l2' : 41.5,
    #'min_sum_hessian_in_leaf' : 0.3,
    'max_bin': 64,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.27,
    'verbose': 0
}


kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):       
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    print('prepare train')
    X_tr = np.hstack([
        X_tr, train_mat[train_index]
    ])
    print('prepare valid')
    X_va = np.hstack([
        X_va, train_mat[test_index]
    ])
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = np.hstack([
        X_te, test_mat
    ])
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 0
prepare train
prepare valid
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's auc: 0.609881
[100]	valid_0's auc: 0.626785
[150]	valid_0's auc: 0.636711
[200]	valid_0's auc: 0.645291
[250]	valid_0's auc: 0.653108
[300]	valid_0's auc: 0.659199
[350]	valid_0's auc: 0.66384
[400]	valid_0's auc: 0.667295
[450]	valid_0's auc: 0.670719
[500]	valid_0's auc: 0.67305
[550]	valid_0's auc: 0.674532
[600]	valid_0's auc: 0.676042
[650]	valid_0's auc: 0.677376
[700]	valid_0's auc: 0.678354
[750]	valid_0's auc: 0.679278
[800]	valid_0's auc: 0.679786
[850]	valid_0's auc: 0.680553
[900]	valid_0's auc: 0.681362
[950]	valid_0's auc: 0.681892
[1000]	valid_0's auc: 0.68238
[1050]	valid_0's auc: 0.682756
[1100]	valid_0's auc: 0.683065
[1150]	valid_0's auc: 0.68333
[1200]	valid_0's auc: 0.683592
[1250]	valid_0's auc: 0.683563
[1300]	valid_0's auc: 0.683724
[1350]	valid_0's auc: 0.683775
[1400]	valid_0's auc: 0.683777
[1450]	valid_0's auc: 0.683716
[1500]	valid_0's auc: 0.683

In [ ]:
model_name = 'tfifg_svd_5folds'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()